### Data prepration and analysis to define metrics

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import the modefied saved new dataset with a new format (parquet)
events = pd.read_parquet('./events.parquet.gz')

In [3]:
events.head()

,event_time,event_type,product_id,category_id,subcategory,brand,price,user_id,user_session,corrected_category,category,corrected_brand,product_name
0,2019-11-01 00:00:14+00:00,cart,1005014,2053013555631882655,electronics.smartphone,samsung,503.09,533326659,6b928be2-2bce-4640-8296-0efdf2fda22a,electronics.smartphone,electronics,None,1005014 - samsung electronics.smartphone
1,2019-11-01 00:00:41+00:00,purchase,13200605,2053013557192163841,furniture.bedroom.bed,None,566.30,559368633,d6034fa2-41fb-4ac0-9051-55ea9fc9147a,None,furniture,None,13200605 - furniture.bedroom.bed
2,2019-11-01 00:01:04+00:00,purchase,1005161,2053013555631882655,electronics.smartphone,xiaomi,211.92,513351129,e6b7ce9b-1938-4e20-976c-8b4163aea11d,electronics.smartphone,electronics,None,1005161 - xiaomi electronics.smartphone
3,2019-11-01 00:03:24+00:00,cart,1801881,2053013554415534427,appliances.personal.massager,samsung,488.80,557746614,4d76d6d3-fff5-4880-8327-e9e57b618e0e,appliances.personal.massager,appliances,None,1801881 - samsung appliances.personal.massager
4,2019-11-01 00:03:39+00:00,cart,1005115,2053013555631882655,electronics.smartphone,apple,949.47,565865924,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,electronics.smartphone,electronics,None,1005115 - apple electronics.smartphone


In [4]:
events.shape

(7033125, 13)

In [5]:
# make an new sub-dataset for only purchases event
purchases = events[events["event_type"] == "purchase"].copy()

In [6]:
# make an new sub-dataset for only cart event
carts = events[events["event_type"] == "cart"].copy()

#### Define league_table: We create new indicator columns to calculate monthly values (product-level table):
#### november_volume col: How many times each product sold in November.
#### november_revenue col: The total prices for each product in november.
#### november_users col: Number of unique users who purchase for each product.
#### The same previous columns repeated for December.

In [7]:

purchases_league_table = ( purchases.assign(november_count=np.where(purchases["event_time"].dt.month==11, 1, 0),\
                                          november_revenue=np.where(purchases["event_time"].dt.month==11, purchases["price"], 0),\
                                          november_user_id=np.where(purchases["event_time"].dt.month==11, purchases["user_id"], np.nan),\
                                          december_count=np.where(purchases["event_time"].dt.month==12, 1, 0),\
                                          december_revenue=np.where(purchases["event_time"].dt.month==12, purchases["price"], 0),\
                                          december_user_id=np.where(purchases["event_time"].dt.month==12, purchases["user_id"], np.nan))\
                          .groupby(["product_id", "product_name"])\
                          .agg(november_volume=('november_count', 'sum'),
                               november_revenue=('november_revenue', 'sum'),
                               november_users=('november_user_id', 'nunique'),
                               december_volume=('december_count', 'sum'),
                               december_revenue=('december_revenue', 'sum'),
                               december_users=('december_user_id', 'nunique')
                              )
                          .assign(
                              volume_diff=lambda x:x["december_volume"] - x["november_volume"],
                              revenue_diff=lambda x:x["december_revenue"] - x["november_revenue"],
                              users_diff=lambda x:x["december_users"] - x["november_users"])
                          .reset_index()
)                                         

In [8]:
purchases_league_table.head()

,product_id,product_name,november_volume,november_revenue,november_users,december_volume,december_revenue,december_users,volume_diff,revenue_diff,users_diff
0,1000978,1000978 - samsung electronics.smartphone,20,6135.32,17,16,4260.40,16,-4,-1874.92,-1
1,1001588,1001588 - meizu electronics.smartphone,6,766.29,5,13,1652.55,12,7,886.26,7
2,1001605,1001605 - apple electronics.smartphone,0,0.00,0,18,9806.76,16,18,9806.76,16
3,1001606,1001606 - apple electronics.smartphone,0,0.00,0,11,5662.69,10,11,5662.69,10
4,1001618,1001618 - apple electronics.smartphone,36,18059.76,25,7,4745.84,7,-29,-13313.92,-18


In [9]:
# conversion table include the number of rows per event type for each product ID per month.
conversion_table = (
    pd.pivot_table(
        data=events.assign(month=events["event_time"].dt.month),
        index=["product_id", "product_name"],
        columns=["month", "event_type"],
        values="user_session",
        aggfunc="count").fillna(0)
    .set_axis(labels=["november_cart", "november_sold",
                      "december_cart", "december_sold"],
              axis=1).reset_index().assign(
        november_conversion=lambda x: x["november_sold"] / x["november_cart"],
        december_conversion=lambda x: x["december_sold"] / x["december_cart"])
)

In [10]:
conversion_table.head()

,product_id,product_name,november_cart,november_sold,december_cart,december_sold,november_conversion,december_conversion
0,1000894,1000894 - texet electronics.smartphone,0.0,0.0,4.0,0.0,NaN,0.000000
1,1000978,1000978 - samsung electronics.smartphone,60.0,20.0,58.0,16.0,0.333333,0.275862
2,1001588,1001588 - meizu electronics.smartphone,16.0,6.0,37.0,13.0,0.375000,0.351351
3,1001605,1001605 - apple electronics.smartphone,0.0,0.0,42.0,18.0,NaN,0.428571
4,1001606,1001606 - apple electronics.smartphone,0.0,0.0,23.0,11.0,NaN,0.478261


In [11]:
# merge purchases_league_table with conversion table based on product_id and product_name.
league_table = purchases_league_table.merge(conversion_table,
                                            on=["product_id", "product_name"], how="left")

In [12]:
league_table.dtypes

product_id               int64
product_name            object
november_volume          int32
november_revenue       float64
november_users           int64
december_volume          int32
december_revenue       float64
december_users           int64
volume_diff              int32
revenue_diff           float64
users_diff               int64
november_cart          float64
november_sold          float64
december_cart          float64
december_sold          float64
november_conversion    float64
december_conversion    float64
dtype: object

In [13]:
league_table.head()

,product_id,product_name,november_volume,november_revenue,november_users,december_volume,december_revenue,december_users,volume_diff,revenue_diff,users_diff,november_cart,november_sold,december_cart,december_sold,november_conversion,december_conversion
0,1000978,1000978 - samsung electronics.smartphone,20,6135.32,17,16,4260.40,16,-4,-1874.92,-1,60.0,20.0,58.0,16.0,0.333333,0.275862
1,1001588,1001588 - meizu electronics.smartphone,6,766.29,5,13,1652.55,12,7,886.26,7,16.0,6.0,37.0,13.0,0.375000,0.351351
2,1001605,1001605 - apple electronics.smartphone,0,0.00,0,18,9806.76,16,18,9806.76,16,0.0,0.0,42.0,18.0,NaN,0.428571
3,1001606,1001606 - apple electronics.smartphone,0,0.00,0,11,5662.69,10,11,5662.69,10,0.0,0.0,23.0,11.0,NaN,0.478261
4,1001618,1001618 - apple electronics.smartphone,36,18059.76,25,7,4745.84,7,-29,-13313.92,-18,69.0,36.0,17.0,7.0,0.521739,0.411765


In [15]:
# month-on-month change in volume
league_table["volume_diff_pct"] = 100 * (league_table["volume_diff"]/league_table["november_volume"])

In [18]:
product_catalog = (
    events[["product_id", "product_name", "category_id", "subcategory", "brand", "category"]]
    .drop_duplicates(subset=["product_id", "product_name", "subcategory", "brand"])
    )

In [19]:
assert len(product_catalog) == events["product_id"].nunique()

In [20]:
print(product_catalog.shape)
product_catalog.head()

(83838, 6)


,product_id,product_name,category_id,subcategory,brand,category
0,1005014,1005014 - samsung electronics.smartphone,2053013555631882655,electronics.smartphone,samsung,electronics
1,13200605,13200605 - furniture.bedroom.bed,2053013557192163841,furniture.bedroom.bed,None,furniture
2,1005161,1005161 - xiaomi electronics.smartphone,2053013555631882655,electronics.smartphone,xiaomi,electronics
3,1801881,1801881 - samsung appliances.personal.massager,2053013554415534427,appliances.personal.massager,samsung,appliances
4,1005115,1005115 - apple electronics.smartphone,2053013555631882655,electronics.smartphone,apple,electronics


### Find the top high-demand products based-on the following metrcis using the previous created tables

In [22]:
# defining cutoff to show the top product performers
# we define the cutoffs that mean we only consider products 

DEC_VS_NOV_PCT_CUTOFF = 200  # Sold at least twice as many in December as November (200%)
NOV_VOLUME_CUTOFF = 10  # # Sold at least 10 in November
ONLY_DEC_VOLUME_CUTOFF = 100  # # Sold at least 100 in December

december_high_performers = (
    pd.concat(
        [
            league_table[(np.isinf(league_table["volume_diff_pct"]) == False) &
            (league_table["november_volume"] > NOV_VOLUME_CUTOFF) &
            (league_table["volume_diff_pct"] > DEC_VS_NOV_PCT_CUTOFF)],
            league_table[(np.isinf(league_table["volume_diff_pct"]))
                & (league_table["december_volume"] > ONLY_DEC_VOLUME_CUTOFF)]
        ], axis = 0, ignore_index=True).merge(product_catalog.drop(columns="product_name"), on="product_id")
)

In [23]:
print(december_high_performers.shape)

(449, 22)


In [24]:
# top 5 products high preformers

print(december_high_performers.head())

   product_id                              product_name  november_volume  \
0     1002527    1002527 - apple electronics.smartphone               14   
1     1003533  1003533 - samsung electronics.smartphone              123   
2     1003712  1003712 - samsung electronics.smartphone              519   
3     1003770   1003770 - huawei electronics.smartphone               11   
4     1003801    1003801 - apple electronics.smartphone              107   

   november_revenue  november_users  december_volume  december_revenue  \
0          10086.08              14              455         303348.41   
1          53570.29             107              450         138198.78   
2         309958.00             406             1806         920216.77   
3           3329.24              10               55          12116.08   
4          70989.71              89              333         208191.34   

   december_users  volume_diff  revenue_diff  ...  november_sold  \
0             300          441

In [25]:
from IPython.display import display

for col in ["category", "subcategory", "brand"]:
    print(col)
    display(december_high_performers[col].value_counts())

category


category
apparel         121
appliances      102
electronics      85
computers        36
furniture        32
kids             24
construction     20
sport            13
auto              8
country_yard      4
medicine          2
accessories       1
stationery        1
Name: count, dtype: int64

subcategory


subcategory
electronics.smartphone               43
apparel.shoes                        37
appliances.kitchen.coffee_grinder    33
apparel.shoes.sandals                28
appliances.personal.massager         26
                                     ..
electronics.audio.microphone          1
apparel.shoes.ballet_shoes            1
kids.dolls                            1
appliances.personal.hair_cutter       1
stationery.cartrige                   1
Name: count, Length: 76, dtype: int64

brand


brand
lucente    56
xiaomi     31
sony       24
samsung    20
huawei     18
           ..
babyzen     1
palit       1
galaxy      1
gorenje     1
saeshin     1
Name: count, Length: 128, dtype: int64

In [26]:
december_high_performers[["category", "subcategory", "brand"]].value_counts().head()

category     subcategory                        brand  
appliances   appliances.kitchen.coffee_grinder  lucente    33
electronics  electronics.smartphone             xiaomi     12
apparel      apparel.shoes                      sony       11
kids         kids.toys                          lucente    10
electronics  electronics.smartphone             huawei      9
Name: count, dtype: int64

### Final conclusion: We have about 450 products you should promote for Christmas period 
### Top high-demand categories are apparel, appliances, and electronics in Christmas period